# Verify quantum program by MorphQPV

**Author:** Debin Xiang & Siwei Tan

**Date:** 12/4/2024

Based on paper "[MorphQPV: Exploiting Isomorphism in Quantum Programs to Facilitate Confident Verification][1]" (ASPLOS 2024)

[1]: https://dl.acm.org/doi/10.1145/3620666.3651360

MorphQPV is a tool to facilitate confident assertion-based verification in quantum computing. It provides a framework for analyzing and verifying quantum circuits using a new type of formalism. It defines an assertion statement that consists of assume-guarantee primitives and tracepoint pragma to label the target quantum state. Then, we can characterize the ground-truth relation between states using isomorphism-based approximation, which can effectively get the program states under various inputs while avoiding repeated executions. Finally, the verification is formulated as a constraint optimization problem with a confidence estimation model to enable rigorous analysis. 


<div style="text-align:center;">
    <img src="../picture/3_1_morphqpv_overview.png"  width="90%" height="90%">
</div>


The figure above presents the verification workflow of MorphQPV
to verify quantum program, consisting of three steps:

Step 1. assertion statement. We label the states in the program via tracepoint pragma. Subsequently, each tracepoint records the time and the associated qubits, which is used to
describe the expected program behavior with these states.
We define an assume-guarantee assertion with predicates to
specify: 
(a) the ranges of these states, represented as objective
functions for each state, e.g., 𝑃1 (𝜌T1 ) and 𝑃2 (𝜌T2 ) in Figure 2;
and (b) the relation between these states, represented as a
objective function involving multiple states, e.g., 𝑃3 (𝜌T1 , 𝜌T2 ).
The predicate is validated on the classical computers.

Step 2. program characterization. MorphQPV characterizes
the natural relations between quantum states by running the
program on the quantum hardware. The characterization
begins with a one-shot input sampling to record the labeled
states across different inputs. By exploiting isomorphism, it
then builds approximation functions based on the sampling
results, e.g., 𝜌T1 = 𝑓1 (𝜌in), representing the relations between
the input and the labeled states. These approximation functions can be efficiently computed to obtain tracepoint states on classical computers.

Step 3. assertion validation. MorphQPV validates the assertion by checking whether the relations in the program satisfy the expected constraints in the assertion. Instead of
testing tremendous inputs to identify the error, we apply a global search that packs the predicates and the approximation functions into a constraint maximization problem.
The assume-guarantee assertion is true only if the maximum objective is less than 0. When the program is incorrect, the maximum argument 𝜌_in is the counter-example resulting in
the bug. When the program is correct, MorphQPV estimates
the confidence based on the accuracy of the characterization.

In [1]:
import os
os.chdir("../..")
import logging
logging.basicConfig(level=logging.WARN)

import numpy as np

from janusq.verification.morphqpv import MorphQC,Config
from janusq.verification.morphqpv import IsPure,Equal,NotEqual
from janusq.verification.morphqpv import StateVector,Expectation
from janusq.verification.morphqpv import pauliX,pauliY,pauliZ,hadamard

## Example of using MorphQPV

### verify a quantum block

In [2]:
myconfig = Config()
myconfig.solver = 'sgd' ## set the stochastic gradient descent method to solve the assertion
with MorphQC(config=myconfig) as morphQC:
    ### morphQC is a quantum circuit, the gate is applyed to the qubits in the order of the list
    ## we can add tracepoint to label the quantum state
    morphQC.add_tracepoint(0,1) ## the state after the first 3 qubits is labeled as tracepoint 0
    morphQC.assume(0,IsPure()) ## the state in tracepoint 0 is assumed to be pure
    morphQC.assume(0,Equal(Expectation(pauliX@pauliY)),0.4)
    morphQC.x([1,3]) ## apply x gate to  qubit 1 and 3
    morphQC.y([0,1,2])  ## apply y gate to qubit 0,1,2
    for i in range(4):
        morphQC.cnot([i, i+1]) ## apply cnot gate to qubit i and i+1
    morphQC.s([0,2,4]) ## apply s gate to qubit 0,2,4
    morphQC.add_tracepoint(2,4) ## the state after qubit 2 and 4 is labeled as tracepoint 1
    morphQC.assume(1,IsPure())  ## the state in tracepoint 1 is assumed to be pure
    morphQC.rz([0,1,2,3,4],np.pi/3) ## apply rz gate to qubit 0,1,2,3,4
    morphQC.h([0,1,2,3,4]) ## apply h gate to qubit 0,1,2,3,4
    morphQC.rx([0,1,2,3,4],np.pi/3) ## apply rx(pi/3) gate to qubit 0,1,2,3,4
    morphQC.ry([0,1,2,3,4],np.pi/3) ## apply ry(pi/3) gate to qubit 0,1,2,3,4
    morphQC.add_tracepoint(0,3) ## the state after qubit 0 and 3 is labeled as tracepoint 2
    morphQC.assume(2,IsPure()) ## the state in tracepoint 2 is assumed to be pure
    morphQC.assume([0,2],Equal(Expectation(pauliX@pauliY)),)
    morphQC.guarantee([1,2],Equal()) ## the state in tracepoint 1 and 2 are guaranteed to be equal
    morphQC.guarantee([0,1],NotEqual()) ## the state in tracepoint 0,1 and 2 are guaranteed to be different
print(morphQC.assertion) ## print the assertion statement and verify result


producing output states:   0%|          | 0/8 [00:00<?, ?it/s]/Users/chutianyao/anaconda3/envs/janusq/lib/python3.10/site-packages/pennylane/_qubit_device.py:653: UserWarning: Requested state or density matrix with finite shots; the returned state information is analytic and is unaffected by sampling. To silence this warning, set shots=None on the device.
  warnings.warn(
sgd optimizing converge:  50%|█████     | 504/1000 [02:03<02:01,  4.07it/s, loss=32.326332, min_loss=29.750893]


{'assume': [([0], IsPure, ()), ([0], Equal, (0.4,)), ([1], IsPure, ()), ([2], IsPure, ()), ([0, 2], Equal, ())], 'gurrantee': [([1, 2], Equal, ()), ([0, 1], NotEqual, ())], 'verify': {'optimal_input_state': Array([-0.56040865-0.17974281j,  0.0841108 -0.6066261j ,
       -0.11731254-0.40672597j, -0.0988518 +0.29853615j], dtype=complex64), 'optimal_gurrantee_value': Array(0.30231148, dtype=float32), 'is_assume_satisfied': [Array(0.04810691, dtype=float32), Array(27.236969, dtype=float32), Array(0.04923344, dtype=float32), Array(0.14673471, dtype=float32), Array(0.3957154, dtype=float32), Array(0.7049782, dtype=float32), Array(0.30231148, dtype=float32)]}}
